# 文本生成

## 说明
本文展示了如何使用appbuilder内置组件以及如何快速构建全新的文本生成组件。以下是一个在营销场景下生成各种文本内容的样例。

## 概览
具体地，本样例按照以下流程进行文本生成：
1. 使用**空模板**构建一个全新的文本生成组件：商品信息生成组件。
2. 输入一个商品，使用新构建的商品信息生成组件生成该商品的商品信息。
3. 基于生成的商品信息，使用内置的**问答对挖掘**组件生成问答对。生成的问答对可用于客服等。
4. 基于生成的商品信息，使用内置的**风格写作**组件为该商品生成一份【小红书】文案。
5. 使用内置的**标签抽取**组件对生成的文案进行标签抽取。抽取出的标签可用于检索文案。

以下我们会演示如何实现我们自己的定制化能力。

## 流程演示
### Step 0: 安装Python SDK
我们的appbuilder支持使用pip安装（要求Python >= 3.8）：

In [ ]:
pip install appbuilder-sdk

代码中需要配置用户的`APPBUILDER_TOKEN`。

In [1]:
import os

# 设置环境变量
os.environ['APPBUILDER_TOKEN'] = '...'

安装成功后，我们就可以搭建我们自己的能力了。

### Step 1: 商品信息生成
基于appbuilder提供的**空模板**，用户可以自定义文本生成组件以解决多样化的需求。这里我们构建一个**商品信息生成**组件以生成商品信息：

In [ ]:
from appbuilder import Message, Playground

# 输入到大模型中的prompt的模板
prompt_template = \
'''输入商品名，我需要你为我生成该商品的商品信息。

要求：
- 你生成的商品信息应该包含多方面信息。
- 开头和结尾不需要有其他与商品信息无关的内容。

商品名：{product_name}
商品信息：
'''
# 创建商品信息生成组件
product_information_generation = Playground(prompt_template=prompt_template, model='ERNIE Speed-AppBuilder')

# 获取商品信息
# 填充prompt_template参数的参数映射表，需要与prompt_template对应
prompt_template_kwargs = {
    'product_name': '特斯拉Model Y'
}
response = product_information_generation(Message(prompt_template_kwargs), stream=False, temperature=0.5)
product_information = response.content
print(f'商品信息：\n{product_information}')

输出结果：

```
商品信息：
特斯拉Model Y是一款电动汽车，它拥有许多独特的功能和特性。以下是它的主要特点和优势：

特斯拉Model Y是一款电动汽车，它采用纯电力驱动，没有排放和噪音污染，具有零启动加速、零延迟、零维护的特性。
特斯拉Model Y拥有先进的自动驾驶技术，可以自动识别交通状况并做出相应的反应，大大提高了驾驶安全性。
特斯拉Model Y的电池寿命长，充电效率高，可以在短时间内将电池充满，同时支持快速充电和慢充两种方式。
特斯拉Model Y的外观设计独特，线条流畅，充满未来感，内部空间宽敞，乘坐舒适。
特斯拉Model Y的价格相对较高，但性价比高，是一款非常值得购买的汽车。

总之，特斯拉Model Y是一款集未来科技、安全性、舒适性和实用性于一体的电动汽车。
```

### Step 2: 问答对生成
基于Step 1生成的商品信息我们生成一些问答对，这里我们使用内置的**问答对生成**组件。

In [ ]:
from appbuilder import QAPairMining

# 初始化问答对生成组件
qa_pair_mining = QAPairMining(model='ERNIE Speed-AppBuilder')

# 获取问答对
response = qa_pair_mining(Message(product_information), stream=False, temperature=1e-10)
qa_pairs = response.content
print(f'问答对：\n{qa_pairs}')

输出结果：

```
问答对：
问题：特斯拉Model Y有什么特点？
答案：特斯拉Model Y是一款电动汽车，它采用纯电力驱动，没有排放和噪音污染，具有零启动加速、零延迟、零维护的特性。

问题：特斯拉Model Y有什么优势？
答案：特斯拉Model Y拥有先进的自动驾驶技术，可以自动识别交通状况并做出相应的反应，大大提高了驾驶安全性。

问题：特斯拉Model Y的电池寿命如何？
答案：特斯拉Model Y的电池寿命长，充电效率高，可以在短时间内将电池充满，同时支持快速充电和慢充两种方式。

问题：特斯拉Model Y的外观设计如何？
答案：特斯拉Model Y的外观设计独特，线条流畅，充满未来感，内部空间宽敞，乘坐舒适。

问题：特斯拉Model Y的价格是多少？
答案：特斯拉Model Y的价格相对较高，但性价比高，是一款非常值得购买的汽车。
```

### Step 3: 营销文案生成
基于Step 1生成的商品信息生成【小红书】文案，这里我们使用内置的**风格写作**组件。

In [ ]:
from appbuilder import StyleWriting

# 构造query
query = \
f'''请你基于以下商品信息生成文案：
{product_information}
'''

# 初始化风格写作组件
style_writing = StyleWriting(model='ERNIE Speed-AppBuilder')

# 获取小红书文案
response = style_writing(Message(query), style_query='小红书', length=300)
copywriting = response.content
print(f'文案：\n{copywriting}')

输出结果：

```
【特斯拉Model Y，一款值得拥有的未来科技之车！🚀】

🚗Model Y作为一款电动汽车，具有许多独特的功能和特性。它采用纯电力驱动，没有排放和噪音污染，具有零启动加速、零延迟、零维护的特性。在城市道路上行驶，它就像是一阵清风，让你轻松应对各种交通状况。💨

🚗特斯拉Model Y还拥有先进的自动驾驶技术，可以自动识别交通状况并做出相应的反应，大大提高了驾驶安全性。在行驶过程中，它就像是一位智能助手，为你提供安全、舒适的驾驶体验。🛡️

🚗Model Y的电池寿命长，充电效率高，可以在短时间内将电池充满，同时支持快速充电和慢充两种方式。无论是长途旅行还是城市代步，它都能满足你的需求。⚡

🚗Model Y的外观设计独特，线条流畅，充满未来感，内部空间宽敞，乘坐舒适。无论是驾驶还是乘坐，都能感受到它的豪华与舒适。🌟

💰虽然价格相对较高，但性价比高，是一款非常值得购买的汽车。它不仅是一款车，更是一种生活方式的体现。🚗

#未来科技 #特斯拉 #安全性 #舒适性 #实用性 #电动汽车
```

### Step 4: 标签抽取
基于Step 3生成的文案进行标签抽取，这里我们使用内置的**标签抽取**组件。

In [ ]:
from appbuilder import TagExtraction

# 初始化标签抽取组件
tag_extraction = TagExtraction(model='ERNIE Speed-AppBuilder')

# 获取标签
response = tag_extraction(Message(copywriting), stream=False, temperature=1e-10)
tags = response.content
print(f'标签：\n{tags}')

输出结果：

```
标签：
1.特斯拉
2.Model Y
3.未来科技
4.纯电力驱动
5.自动驾驶
6.电池寿命
7.充电效率
8.外观设计
9.内部空间
10.性价比
```